In [124]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Mar 25 20:34:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [125]:
!git clone https://gitlab.com/bigirqu/quranqa.git
!pip install git+https://github.com/neuml/txtai datasets pandas
!pip install farasapy

fatal: destination path 'quranqa' already exists and is not an empty directory.
  Cloning https://github.com/neuml/txtai to /tmp/pip-req-build-tsjqnk85
  Running command git clone -q https://github.com/neuml/txtai /tmp/pip-req-build-tsjqnk85
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [126]:
import sys
sys.path.insert(1, '/content/quranqa/code')

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from txtai.pipeline import HFTrainer
import torch, string, re, os, json
import operator

from farasa.pos import FarasaPOSTagger 
from farasa.ner import FarasaNamedEntityRecognizer 
from farasa.diacratizer import FarasaDiacritizer 
from farasa.segmenter import FarasaSegmenter 
from farasa.stemmer import FarasaStemmer

import read_write_qrcd as q_reader
from gensim import corpora, models, similarities
from collections import defaultdict
import pprint

In [127]:
train_passage_question_objects = q_reader.load_jsonl('./quranqa/datasets/qrcd_v1.1_train.jsonl')
dev_passage_question_objects = q_reader.load_jsonl('./quranqa/datasets/qrcd_v1.1_dev.jsonl')


Loaded 710 records from ./quranqa/datasets/qrcd_v1.1_train.jsonl
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl


In [128]:

train_data = [dict({"pq_id": passage_question_object['pq_id'],
                    #"question": quranqa22_eval.normalize_text(passage_question_object.question),
                    "question": passage_question_object['question'],
                    "passage": passage_question_object['passage'], 
                    "answers": r['text']})
              for passage_question_object in train_passage_question_objects
              for r in passage_question_object['answers']]


dev_data = [dict({"pq_id": passage_question_object['pq_id'],
                  #"question": quranqa22_eval.normalize_text(passage_question_object.question),
                  "question": passage_question_object['question'],
                  "passage": passage_question_object['passage'],
                  "answers": r['text']})
            for passage_question_object in dev_passage_question_objects
            for r in passage_question_object['answers']]



In [129]:
import pandas as pd
dev_data_df = pd.DataFrame(dev_data)
dev_data_df.to_csv('dev.csv', sep='\t')
train_data_df = pd.DataFrame(train_data)
train_data_df.to_csv('train.csv', sep='\t')

!cp *.csv /content/drive/MyDrive

cp: target '/content/drive/MyDrive' is not a directory


In [130]:
pretrained_models = [#"wissamantoun/araelectra-base-artydiqa", #num_train_epochs=2
                     #"salti/AraElectra-base-finetuned-ARCD", #num_train_epochs=8
                     "gfdgdfgdg/arap_qa_bert_large_v2",
                     #"gfdgdfgdg/arap_qa_bert_v2",
                     #"gfdgdfgdg/arap_qa_bert"
                     ]


def tokenize_function(row,tokenizer):
    return tokenizer.encode_plus(row['question'], row['passage'], 
                                 return_tensors='pt', padding=True, 
                                 truncation=True,max_length=512, 
                                 add_special_tokens = True)

topk_n = 20
def get_prediction(row, model, tokenizer):
    inputs = tokenize_function(row,tokenizer)
    output = model(**inputs)
    
    #print(torch.topk(output.start_logits.flatten(), topk_n))
    answer_starts_val = torch.topk(output.start_logits.flatten(), topk_n).values
    answer_ends_val = torch.topk(output.end_logits.flatten(), topk_n).values
    answer_starts = torch.topk(output.start_logits.flatten(), topk_n).indices
    answer_ends = torch.topk(output.end_logits.flatten(), topk_n).indices
    
    answers = []
    rank = 1
    for answer_start,answer_end,answer_start_v,answer_end_v in zip(answer_starts,answer_ends,answer_starts_val,answer_ends_val):
        answer = {}
        ans = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end+1]))
        if len(ans.strip())>0:
            answer["answer"] = ans
            answer["rank"] = rank
            answer["score"] = answer_start_v.item()+answer_end_v.item()
            rank = rank +1
            answers.append(answer)
        if rank == 6:
            break
        #answer = str(answer_start_v.item())+' '+ answer +' '+ str(answer_end_v.item())
    return answers


In [131]:

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    print(tokenizer.decode(prediction[0]))
    print(tokenizer.decode(prediction))
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    exact_match = compute_exact_match(pred, labels)
    f1 = compute_f1(pred, labels)

    return { "f1": f1, "exact_match": exact_match}


In [132]:
from transformers import EarlyStoppingCallback

def Fine_tune(model_name,traindata,devdata,num_train_epochs,batch_size):
    trainer = HFTrainer()

    return trainer(model_name, traindata,#devdata,
                   task="question-answering",
                   metric_for_best_model = 'f1',
                   #evaluation_strategy ='epoch',
                   #eval_steps = 500, # Evaluation and Save happens every 50 steps
                   #save_total_limit = 15,
                   save_strategy='no',
                   learning_rate=2e-5,
                   per_device_train_batch_size=batch_size,
                   #per_device_eval_batch_size=batch_size,
                   num_train_epochs=num_train_epochs,
                   weight_decay=0.01,
                   #load_best_model_at_end=False, # change this to True after hyperparameter tuning
                   )
    #return trainer(model_name, data, task="question-answering", num_train_epochs=num_train_epochs)#num_train_epochs=10

In [138]:
from gensim import corpora, models, similarities
from collections import defaultdict
import pprint
def ir(data_item={}, query_key='question', docs_key='passage', index_method='lsi', dims=4) -> list:
    # get data items 
    docs = [ver.strip() for ver in data_item.get(docs_key).split('.') if ver.strip()]
    query = data_item.get(query_key)
    stoplist = ''.split()
    texts = [
        [word for word in doc.split() if word not in stoplist]
        for doc in docs
    ]

    # build dictionary and corpus 
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    # build tfidf & lsi models 
    tfidf = models.TfidfModel(corpus) 
    corpus_tfidf = tfidf[corpus]
    lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=dims)
    corpus_lsi = lsi_model[corpus_tfidf]

    # generate query BOW vector and LSI vector from the question 
    query_bow = query.replace('؟', '').replace('"', '').split()
    vec_bow = dictionary.doc2bow(query_bow)
    vec_lsi = lsi_model[vec_bow]

    result = []
    try:
        # generate tfidf and lsi indices 
        lsi_index = similarities.MatrixSimilarity(corpus_lsi)
        tfidf_index = similarities.MatrixSimilarity(corpus_tfidf)
        
        
        # get similarities between queries and docs 
        tfidf_sims = tfidf_index[vec_bow]
        lsi_sims = lsi_index[vec_lsi]

        # sort documents according to similarties 
        
        if index_method == 'tfidf':
            sims = tfidf_sims
        else:
            sims = lsi_sims
        
        sims = sorted(enumerate(sims), key=lambda item: -item[1])
        for doc_position, doc_score in sims:
            result.append( (doc_score, docs[doc_position]) )
    except:
        for doc in docs:
            result.append( (0.0, doc) )
    return result

In [139]:
def filter_passage(in_data):
  f_train_data = []
  for data in in_data:
    filtered = ir(data_item=data)
    sc,worest = filtered[-1]
    passage = data['passage']
    if sc != 0:
      passage = passage.replace(worest,'')
    passage = passage.replace('  ',' ')
    passage = passage.replace(' .','.')
    passage = passage.replace('..','.')
    data['passage'] = passage
    f_train_data.append(data)
  return f_train_data

In [ ]:
f_train_data = filter_passage(train_data)
f_dev_data = filter_passage(dev_data)


In [143]:
def test_model(tokenizer,model,data):
    result = {}
    for row in data:
        pred = get_prediction(row, model, tokenizer)
        result[row["pq_id"]] = pred
    return result

In [144]:
model_name = "salti/AraElectra-base-finetuned-ARCD" #num_train_epochs=11
num_train_epochs=11
batch_size=1

print("\n\n###################"+model_name)
tokenizer_salti = AutoTokenizer.from_pretrained(model_name)
model_salti = AutoModelForQuestionAnswering.from_pretrained(model_name)
result = test_model(tokenizer_salti,model_salti,dev_data)
open('./quranqa/datasets/salti_RunID.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/salti_RunID.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'

model, tokenizer = Fine_tune(model_name,train_data,dev_data,num_train_epochs,batch_size)
os.makedirs('salti')
model.save_pretrained('salti')
tokenizer.save_pretrained('salti')
tokenizer = AutoTokenizer.from_pretrained('./salti')
model = AutoModelForQuestionAnswering.from_pretrained('./salti')
result = test_model(tokenizer,model,dev_data)
open('./quranqa/datasets/salti_Run'+str(num_train_epochs)+'F.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/salti_Run{num_train_epochs}F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'



TypeError: ignored

In [ ]:
from txtai.embeddings import Embeddings
from txtai.pipeline import Extractor

# Create embeddings model, backed by sentence-transformers & transformers
embeddings = Embeddings({"path": model_name,"passage":3})

# Create extractor instance
extractor = Extractor(embeddings, model_name)

Some weights of the model checkpoint at salti/AraElectra-base-finetuned-ARCD were not used when initializing ElectraModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from txtai.pipeline import Similarity

# Create and run pipeline
similarity = Similarity()
q = "feel good story"
d = ["Maine man wins $1M from $25 lottery ticket", 
    "Don't sacrifice slower friends in a bear attack"]
similarity(q, d)

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


[(0, 0.6845324039459229), (1, 0.0025504184886813164)]

In [ ]:
from txtai.pipeline import Similarity
cont = 'وإذ ابتلى إبراهيم ربه بكلمات فأتمهن قال إني جاعلك للناس إماما قال ومن ذريتي قال لا ينال عهدي الظالمين. وإذ جعلنا البيت مثابة للناس وأمنا واتخذوا من مقام إبراهيم مصلى وعهدنا إلى إبراهيم وإسماعيل أن طهرا بيتي للطائفين والعاكفين والركع السجود. وإذ قال إبراهيم رب اجعل هذا بلدا آمنا وارزق أهله من الثمرات من آمن منهم بالله واليوم الآخر قال ومن كفر فأمتعه قليلا ثم أضطره إلى عذاب النار وبئس المصير. وإذ يرفع إبراهيم القواعد من البيت وإسماعيل ربنا تقبل منا إنك أنت السميع العليم. ربنا واجعلنا مسلمين لك ومن ذريتنا أمة مسلمة لك وأرنا مناسكنا وتب علينا إنك أنت التواب الرحيم. ربنا وابعث فيهم رسولا منهم يتلو عليهم آياتك ويعلمهم الكتاب والحكمة ويزكيهم إنك أنت العزيز الحكيم.'
q = 'من بنى الكعبة'
qu = ['من بنى الكعبة']
data = cont.split('.')

# Create and run pipeline
similarity = Similarity('./salti')
similarity(q, data)

In [ ]:

from txtai.embeddings import Embeddings
from txtai.pipeline import Extractor

# Create embeddings model, backed by sentence-transformers & transformers
embeddings = Embeddings({"path": "sentence-transformers/nli-mpnet-base-v2"})

# Create extractor instance
extractor = Extractor(embeddings, "distilbert-base-cased-distilled-squad")
data = ["Giants hit 3 HRs to down Dodgers",
        "Giants 5 Dodgers 4 final",
        "Dodgers drop Game 2 against the Giants, 5-4",
        "Blue Jays beat Red Sox final score 2-1",
        "Red Sox lost to the Blue Jays, 2-1",
        "Blue Jays at Red Sox is over. Score: 2-1",
        "Phillies win over the Braves, 5-0",
        "Phillies 5 Braves 0 final",
        "Final: Braves lose to the Phillies in the series opener, 5-0",
        "Lightning goaltender pulled, lose to Flyers 4-1",
        "Flyers 4 Lightning 1 final",
        "Flyers win 4-1"]

questions = ["What team won the game?", "What was score?"]

extractor([(question, question, question, False) for question in questions], data)


[('What team won the game?', 'Braves'), ('What was score?', '2-1')]

In [ ]:
dev_data[0]

{'answers': 'للفقراء والمساكين والعاملين عليها والمؤلفة قلوبهم وفي الرقاب والغارمين وفي سبيل الله وابن السبيل',
 'context': 'إنما الصدقات للفقراء والمساكين والعاملين عليها والمؤلفة قلوبهم وفي الرقاب والغارمين وفي سبيل الله وابن السبيل فريضة من الله والله عليم حكيم. ومنهم الذين يؤذون النبي ويقولون هو أذن قل أذن خير لكم يؤمن بالله ويؤمن للمؤمنين ورحمة للذين آمنوا منكم والذين يؤذون رسول الله لهم عذاب أليم.',
 'pq_id': '9:60-61_316',
 'question': 'ما هي مصارف الزكاة؟'}

In [12]:
from txtai.pipeline import Questions
result ={}
tokenizer = AutoTokenizer.from_pretrained('./salti')
model = AutoModelForQuestionAnswering.from_pretrained('./salti')
questions = Questions(path=(model, tokenizer), gpu=True)
i=0
for row in dev_data:
  data = row['passage'].split('.')
  qu = [row['question']]
  rank = 1
  
  answers = []

  ans = list(set(questions([qu[0]] * len(data), data)))
  for r in ans:
    if r != None:
      answer = {}
      answer["answer"] = r
      answer["rank"] = rank
      answer["score"] = 5 - rank
      rank = rank +1
      answers.append(answer)
    if rank == 6:
        break
        
  if len(answers):
    result[row["pq_id"]] = answers
  i = i+1
  '''if i==18:
    print(answers)
    break'''


open('./quranqa/datasets/Que_Run1F.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
!python ./quranqa/code/quranqa22_eval.py --run_file './quranqa/datasets/Que_Run1F.json' --gold_answers_file './quranqa/datasets/qrcd_v1.1_dev.jsonl'


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:999: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


[2022-03-25 14:48:57,742 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
Loaded 109 records from ./quranqa/datasets/qrcd_v1.1_dev.jsonl
The run file is correct.
Unanswered question 9:122-123_400 will receive score 0.
Unanswered question 39:9-10_313 will receive score 0.
Unanswered question 33:56-59_348 will receive score 0.
Unanswered question 3:164-165_415 will receive score 0.
Unanswered question 48:16-17_415 will receive score 0.
Unanswered question 49:6-8_415 will receive score 0.
{"pRR": 0.32898166805660595, "exact_match": 0.1559633027522936, "f1": 0.28199300445676023}


In [ ]:

result['2:178-179_400']



[{'answer': 'ولكم في القصاص حياة', 'rank': 1, 'score': 4},
 {'answer': 'كتب عليكم القصاص في القتلى الحر بالحر والعبد بالعبد والأنثى بالأنثى',
  'rank': 2,
  'score': 3}]